In [33]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [34]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [35]:
if not IN_COLAB:
  %load_ext autoreload
  %autoreload 2
  %load_ext dotenv
  %dotenv

In [36]:
import sys
import os
from pathlib import Path
if not IN_COLAB:
  print(Path(os.getcwd()).parent.absolute())
  sys.path.append(os.path.dirname(os.getcwd()))
  sys.path.append(Path(os.getcwd()).parent.absolute())
else:
  from google.colab import drive
  drive.mount('/content/drive')
  sys.path.append("/content/drive/MyDrive/Colab Notebooks/MgennPy")
import mcore as mc
from common import *



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
RobotsLogger.default = RobotsLogger()

In [38]:
# Neuron Test
n = mc.Neuron()
n.energyLeak = .1
n.peakEnergy = 2.
for i in range(50):
    n.onSignal(i, .2)
    tick_rc = n.onTick(i)
    print(f"iter({i}) {n} ticks with {tick_rc} result ")

iter(0) N[0]e:0.1 l:0.1 p:2.0 ticks with 0.0 result 
iter(1) N[0]e:0.20000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(2) N[0]e:0.30000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(3) N[0]e:0.4 l:0.1 p:2.0 ticks with 0.0 result 
iter(4) N[0]e:0.5000000000000001 l:0.1 p:2.0 ticks with 0.0 result 
iter(5) N[0]e:0.6000000000000002 l:0.1 p:2.0 ticks with 0.0 result 
iter(6) N[0]e:0.7000000000000003 l:0.1 p:2.0 ticks with 0.0 result 
iter(7) N[0]e:0.8000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(8) N[0]e:0.9000000000000005 l:0.1 p:2.0 ticks with 0.0 result 
iter(9) N[0]e:1.0000000000000004 l:0.1 p:2.0 ticks with 0.0 result 
iter(10) N[0]e:1.1000000000000003 l:0.1 p:2.0 ticks with 0.0 result 
iter(11) N[0]e:1.2000000000000002 l:0.1 p:2.0 ticks with 0.0 result 
iter(12) N[0]e:1.3 l:0.1 p:2.0 ticks with 0.0 result 
iter(13) N[0]e:1.4 l:0.1 p:2.0 ticks with 0.0 result 
iter(14) N[0]e:1.4999999999999998 l:0.1 p:2.0 ticks with 0.0 result 
iter(15) N[0]e:1.5999999999999996

In [39]:
RobotsLogger.default.print()
RobotsLogger.default.clean()

{"t": 1733782693.005863, "e": "NEURON_IN", "src": "Neuron.0.fd92b2a11c6442e2", "args": {"tick": 0, "amp": 0.2, "from": 0, "oid": "0", "ohash": "0.18271863043231335138"}}
{"t": 1733782693.0060036, "e": "NEURON_IN", "src": "Neuron.0.f609f08a3aef9b", "args": {"tick": 1, "amp": 0.2, "from": 0, "oid": "0", "ohash": "0.69253772986740635"}}
{"t": 1733782693.0060575, "e": "NEURON_IN", "src": "Neuron.0.c5225967fb0fbe5c", "args": {"tick": 2, "amp": 0.2, "from": 0, "oid": "0", "ohash": "0.14205014477808582236"}}
{"t": 1733782693.0061066, "e": "NEURON_IN", "src": "Neuron.0.a2da10145e7c2120", "args": {"tick": 3, "amp": 0.2, "from": 0, "oid": "0", "ohash": "0.11734709458737832224"}}
{"t": 1733782693.0061529, "e": "NEURON_IN", "src": "Neuron.0.19479cf74b6168c", "args": {"tick": 4, "amp": 0.2, "from": 0, "oid": "0", "ohash": "0.113849822514386572"}}
{"t": 1733782693.0062037, "e": "NEURON_IN", "src": "Neuron.0.4227b83cf9d51189", "args": {"tick": 5, "amp": 0.2, "from": 0, "oid": "0", "ohash": "0.4766981

In [40]:
l1 = mc.Link()
l1.apt = 0.1
l1.length = 10
for i in range(99):
    l1.onSignal(tick_num = 1, amplitude = 1., from_id=i) ## too small
for i in range(99):
    rc = l1.onTick(i)


l2 = mc.Link()
l2.apt = 0.1
l2.length = 100
initial = 10
for i in range(initial, 99):
    l2.onSignal(tick_num = i, amplitude = (1. + float(i)), from_id=i)
print(l2.events)
for i in range(initial, 99):
    rc = l2.onTick(l2.length+i)
    print(f"t{l2.length+i} expected:{float(i-initial)+1.} act:{rc}")
l2.events

[LinkEvent(from_10. 1.0 at 110), LinkEvent(from_11. 2.0 at 111), LinkEvent(from_12. 3.0 at 112), LinkEvent(from_13. 4.0 at 113), LinkEvent(from_14. 5.0 at 114), LinkEvent(from_15. 6.0 at 115), LinkEvent(from_16. 7.0 at 116), LinkEvent(from_17. 8.0 at 117), LinkEvent(from_18. 9.0 at 118), LinkEvent(from_19. 10.0 at 119), LinkEvent(from_20. 11.0 at 120), LinkEvent(from_21. 12.0 at 121), LinkEvent(from_22. 13.0 at 122), LinkEvent(from_23. 14.0 at 123), LinkEvent(from_24. 15.0 at 124), LinkEvent(from_25. 16.0 at 125), LinkEvent(from_26. 17.0 at 126), LinkEvent(from_27. 18.0 at 127), LinkEvent(from_28. 19.0 at 128), LinkEvent(from_29. 20.0 at 129), LinkEvent(from_30. 21.0 at 130), LinkEvent(from_31. 22.0 at 131), LinkEvent(from_32. 23.0 at 132), LinkEvent(from_33. 24.0 at 133), LinkEvent(from_34. 25.0 at 134), LinkEvent(from_35. 26.0 at 135), LinkEvent(from_36. 27.0 at 136), LinkEvent(from_37. 28.0 at 137), LinkEvent(from_38. 29.0 at 138), LinkEvent(from_39. 30.0 at 139), LinkEvent(from_40.

[]

In [41]:
RobotsLogger.default.print()
RobotsLogger.default.clean()

{"t": 1733782693.1084359, "e": "LINK_IGNORE_SIGNAL", "src": "Link.0.c7a5e88dd01a92f8", "args": {"tick": 1, "amp": 1.0, "from": 0, "oid": "0", "ohash": "0.14386160280484352760"}}
{"t": 1733782693.108449, "e": "LINK_IGNORE_SIGNAL", "src": "Link.0.c7a5e88dd01a92f8", "args": {"tick": 1, "amp": 1.0, "from": 1, "oid": "0", "ohash": "0.14386160280484352760"}}
{"t": 1733782693.108456, "e": "LINK_IGNORE_SIGNAL", "src": "Link.0.c7a5e88dd01a92f8", "args": {"tick": 1, "amp": 1.0, "from": 2, "oid": "0", "ohash": "0.14386160280484352760"}}
{"t": 1733782693.108462, "e": "LINK_IGNORE_SIGNAL", "src": "Link.0.c7a5e88dd01a92f8", "args": {"tick": 1, "amp": 1.0, "from": 3, "oid": "0", "ohash": "0.14386160280484352760"}}
{"t": 1733782693.1084683, "e": "LINK_IGNORE_SIGNAL", "src": "Link.0.c7a5e88dd01a92f8", "args": {"tick": 1, "amp": 1.0, "from": 4, "oid": "0", "ohash": "0.14386160280484352760"}}
{"t": 1733782693.108474, "e": "LINK_IGNORE_SIGNAL", "src": "Link.0.c7a5e88dd01a92f8", "args": {"tick": 1, "amp": 